
# Tutorial 7b: Chọc lọc dữ liệu

Nội dung chính:

* Phương thưc loại bỏ
    - Loại bỏ các đặc trung không liên quan
    - Loại bỏ các dòng không liên quan

* pandas
    - Chọn lọc đơn giản sử dụng pandas
    - Suy luận nội suy sử dụng pandas
    
* sklearn
    - Chọn lọc sử dụng sklearn
    - KNN sử dụng trong sklearn
    - Bổ sung dữ liệu lặp theo sklearn

* Áp dụng mô hình ọc theo dữ liệu test

----

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Nạp và khám phá dữ liệu

In [2]:
import pandas as pd
# Hoặc tải dữ liệu titanic đã được chia thành tập dữ liệu huấn luyện và tập dữ liệu thử nghiệm theo https://www.kaggle.com/c/titanic/data
# Nhưng dữ liệu thử nghiệm của kaggle không có nhãn
# Do đó, chúng ta sẽ tải toàn bộ dữ liệu từ kho dữ liệu rồi sau đó chia nhỏ.
titanic_data = pd.read_csv("https://www.openml.org/data/get_csv/16826755/phpMYEkMl.csv", na_values=['?']) #yo
titanic_data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


**Các giá trị được coi là "bị thiếu"**

Có nhiều cách để biểu diễn các giá trị bị thiếu trong cả tệp tập dữ liệu và python pandas.

Các giá trị bị thiếu trong dữ liệu có thể là các mục trống, hoặc '?', hoặc một ký tự khác mà người thu thập dữ liệu đã thống nhất để biểu diễn dữ liệu chưa được quan sát.
Trong trường hợp này là '?' -- biết được điều này, chúng ta cho `pandas` biết những gì cần được coi là giá trị bị thiếu thông qua `na_values=['?']`.

Ở "đầu kia", `pandas` có thể biểu diễn các giá trị bị thiếu theo nhiều cách khác nhau. Như có thể thấy ở trên, "NaN" là ký hiệu giá trị bị thiếu mặc định, tuy nhiên, chúng ta cần có khả năng dễ dàng phát hiện giá trị này với dữ liệu thuộc các kiểu khác nhau: dấu phẩy động, số nguyên, boolean và đối tượng chung. Tuy nhiên, trong nhiều trường hợp, một số dạng khác có thể tham chiếu đến các giá trị bị thiếu như None "bị thiếu" hoặc "không khả dụng", "NA" hoặc (-)inf.

In [3]:
# Chúng ta hãy bỏ qua một số tính năng mà chúng ta sẽ không xem xét ở đây.
titanic_data.drop(['name','ticket', 'embarked', 'boat' ,'body' ,'home.dest'], axis=1, inplace=True)

Bây giờ chúng ta sẽ chia dữ liệu để đào tạo và kiểm tra các tập hợp con vì **CHỈ** dữ liệu đào tạo sẽ được sử dụng để tìm hiểu các yếu tố đầu vào sau đó các mô hình đã học được sẽ được áp dụng cho dữ liệu kiểm tra

In [4]:
from sklearn.model_selection import train_test_split
y=titanic_data['survived']
X=titanic_data.drop(['survived'], axis=1)
X_titanic_train, X_titanic_test, y_titanic_train, y_titanic_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Có một vấn đề là một số tính năng chứa các giá trị chuỗi, cụ thể là các tính năng "sex" và "cabin", vì vậy hãy mã hóa các tính năng này

In [ ]:
# Chúng ta cần sklearn được nâng cấp để chấp nhận các tham số cho bộ mã hóa
import sklearn
!pip install -U scikit-learn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.3.0.


In [ ]:
import numpy as np
# Mã hóa các đặc điểm phân loại bằng cách giữ nguyên các giá trị bị thiếu trong các đặc điểm không đầy đủ
from sklearn.preprocessing import OrdinalEncoder
encoder_sex = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value=np.nan)
X_titanic_train_encoded=X_titanic_train.copy()
X_titanic_train_encoded['sex'] = encoder_sex.fit_transform(X_titanic_train_encoded['sex'].values.reshape(-1, 1))

# Bây giờ chúng ta hãy mã hóa tính năng Cabin chưa hoàn chỉnh
encoder_cabin = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value=np.nan) #You can use the same encoder for both but we use two for the sake of clarfication
X_titanic_train_encoded['cabin'] = encoder_cabin.fit_transform(X_titanic_train_encoded['cabin'].values.reshape(-1, 1).astype(str))
# Lấy mã giá trị "nan" cho tính năng phân loại cabin
cabin_nan_code=encoder_cabin.transform([['nan']])[0][0]
#print(cabin_nan_code)
# Bây giờ, hãy lấy lại các giá trị nan bị thiếu trong dữ liệu được mã hóa
X_titanic_train_encoded['cabin'].replace(cabin_nan_code,np.nan,inplace=True)


## `X_titanic_train_encoded` là dữ liệu đào tạo không đầy đủ được mã hóa

In [ ]:
# Kiểm tra các loại dữ liệu được mã hóa, không có tính năng đối tượng
X_titanic_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 916 entries, 1214 to 1126
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  916 non-null    int64  
 1   sex     916 non-null    float64
 2   age     729 non-null    float64
 3   sibsp   916 non-null    int64  
 4   parch   916 non-null    int64  
 5   fare    915 non-null    float64
 6   cabin   204 non-null    float64
dtypes: float64(4), int64(3)
memory usage: 57.2 KB


## Lưu ý lỗi:ValueError: Đầu vào chứa NaN, vô cực hoặc giá trị quá lớn đối với dtype('float32').

Chúng ta cần xử lý các giá trị bị thiếu trước khi thực hiện phân loại.

Hãy hiển thị số lượng giá trị bị thiếu trong mỗi tính năng của dữ liệu tàu được mã hóa

In [ ]:
print("The number of missing values ")
print(X_titanic_train_encoded.isnull().sum())

The number of missing values 
pclass      0
sex         0
age       187
sibsp       0
parch       0
fare        1
cabin     712
dtype: int64


Chúng tôi có ba tính năng chưa hoàn chỉnh là "tuổi", "giá vé" và "cabin"

## Phương pháp xóa

### Xóa các tính năng chưa hoàn thiện

In [ ]:
X_titanic_train_complete=X_titanic_train_encoded.copy()
X_titanic_train_complete.dropna(axis=1, inplace=True)
X_titanic_train_complete

,pclass,sex,sibsp,parch
1214,3,1.0,0,0
677,3,1.0,0,0
534,2,0.0,0,0
1174,3,0.0,8,2
864,3,0.0,0,0
...,...,...,...,...
1095,3,0.0,0,0
1130,3,0.0,0,0
1294,3,1.0,0,0
860,3,0.0,0,0


In [ ]:
# Kiểm tra số lượng giá trị bị thiếu
print(X_titanic_train_complete.isnull().sum())

pclass    0
sex       0
sibsp     0
parch     0
dtype: int64


### Xóa các phiên bản chưa hoàn chỉnh

In [ ]:
X_titanic_train_complete=X_titanic_train_encoded.copy()
X_titanic_train_complete.dropna(axis=0, inplace=True)
#Sự khác biệt là trục = 0 thay vì 1
X_titanic_train_complete

,pclass,sex,age,sibsp,parch,fare,cabin
39,1,1.0,48.0,0,0,50.4958,14.0
30,1,1.0,45.0,0,0,35.5000,145.0
242,1,0.0,33.0,0,0,27.7208,0.0
136,1,1.0,53.0,0,0,28.5000,68.0
3,1,1.0,30.0,1,2,151.5500,61.0
...,...,...,...,...,...,...,...
189,1,1.0,29.0,0,0,30.0000,113.0
252,1,1.0,61.0,1,3,262.3750,35.0
21,1,0.0,47.0,1,1,52.5542,101.0
276,1,1.0,57.0,1,0,146.5208,42.0


## Lưu ý sự giảm sút về số lượng trường hợp

Một điểm quan trọng khác đối với phương pháp xóa phiên bản là cần phải xóa các giá trị mục tiêu (từ y_train) tương ứng với các phiên bản dữ liệu chưa hoàn chỉnh (đã xóa).

In [ ]:
# Kiểm tra số lượng giá trị bị thiếu
print(X_titanic_train_complete.isnull().sum())

pclass    0
sex       0
age       0
sibsp     0
parch     0
fare      0
cabin     0
dtype: int64


Phương pháp xóa có một số nhược điểm. Nó làm giảm dữ liệu khả dụng, hạn chế khả năng học, đặc biệt là khi có nhiều giá trị bị thiếu.

Hơn nữa, phương pháp xóa các trường hợp không đầy đủ không thực tế đối với dữ liệu thử nghiệm: chúng ta thực sự muốn biết câu trả lời!

## Quy imputation sử dụng `pandas`

### Phép quy imputation đơn giản (`pandas`)

In [ ]:
# Giá trị trung bình cho các giá trị số
X_titanic_data_complete=X_titanic_train_encoded.copy()
X_titanic_data_complete['age']=X_titanic_data_complete['age'].fillna(X_titanic_data_complete['age'].mean())
X_titanic_data_complete['fare']=X_titanic_data_complete['fare'].fillna(X_titanic_data_complete['fare'].mean())
X_titanic_data_complete['cabin']=X_titanic_data_complete['cabin'].fillna(X_titanic_data_complete['cabin'].mean())
# Hiển thị số lượng giá trị bị thiếu
print(X_titanic_data_complete.isnull().sum())

pclass    0
sex       0
age       0
sibsp     0
parch     0
fare      0
cabin     0
dtype: int64


In [ ]:
X_titanic_data_complete.head()

,pclass,sex,age,sibsp,parch,fare,cabin
1214,3,1.0,29.102309,0,0,8.6625,73.27451
677,3,1.0,26.000000,0,0,7.8958,73.27451
534,2,0.0,19.000000,0,0,26.0000,73.27451
1174,3,0.0,29.102309,8,2,69.5500,73.27451
864,3,0.0,28.000000,0,0,7.7750,73.27451


## "Nội suy" (`pandas`)

In [ ]:
X_titanic_data_complete = X_titanic_train_encoded.copy()
X_titanic_data_complete = X_titanic_data_complete.interpolate()
#Đầu ra là 'numpy.ndarray' nên chúng tôi chuyển đổi nó thành khung dữ liệu để đảm bảo tính nhất quán
X_titanic_train_complete = pd.DataFrame(X_titanic_train_complete)
print(X_titanic_train_complete.isna().sum())

pclass    0
sex       0
age       0
sibsp     0
parch     0
fare      0
cabin     0
dtype: int64


## Quy nạp sử dụng `sklearn`

### Phép quy imputation đơn giản (`sklearn`)

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
X_titanic_train_complete = imputer.fit_transform(X_titanic_train_encoded)
#Đầu ra là 'numpy.ndarray' nên chúng tôi chuyển đổi nó thành khung dữ liệu để đảm bảo tính nhất quán
X_titanic_train_complete=pd.DataFrame(X_titanic_train_complete, columns=X_titanic_train_encoded.columns)
print("The number of missing values :\n", X_titanic_train_complete.isnull().sum())

The number of missing values :
 pclass    0
sex       0
age       0
sibsp     0
parch     0
fare      0
cabin     0
dtype: int64


In [ ]:
X_titanic_train_encoded

,pclass,sex,age,sibsp,parch,fare,cabin
1214,3,1.0,NaN,0,0,8.6625,NaN
677,3,1.0,26.0,0,0,7.8958,NaN
534,2,0.0,19.0,0,0,26.0000,NaN
1174,3,0.0,NaN,8,2,69.5500,NaN
864,3,0.0,28.0,0,0,7.7750,NaN
...,...,...,...,...,...,...,...
1095,3,0.0,NaN,0,0,7.6292,NaN
1130,3,0.0,18.0,0,0,7.7750,NaN
1294,3,1.0,28.5,0,0,16.1000,NaN
860,3,0.0,26.0,0,0,7.9250,NaN


## Chiến lược mặc định cho sklearn simple imputer là "mean", bạn có thể thay đổi nó bằng cách sử dụng tham số strategy

In [ ]:
imputer = SimpleImputer(strategy="median")
X_titanic_train_complete = imputer.fit_transform(X_titanic_train_encoded)
# Đầu ra là 'numpy.ndarray' nên chúng tôi chuyển đổi nó thành khung dữ liệu để đảm bảo tính nhất quán
X_titanic_train_complete=pd.DataFrame(X_titanic_train_complete, columns=X_titanic_train_encoded.columns)
print("The number of missing values :\n", X_titanic_train_complete.isnull().sum())

The number of missing values :
 pclass    0
sex       0
age       0
sibsp     0
parch     0
fare      0
cabin     0
dtype: int64


## kNN imputer (`sklearn`)

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer()
X_titanic_train_complete = imputer.fit_transform(X_titanic_train_encoded)
# Đầu ra là 'numpy.ndarray' nên chúng tôi chuyển đổi nó thành khung dữ liệu để đảm bảo tính nhất quán
X_titanic_train_complete=pd.DataFrame(X_titanic_train_complete, columns=X_titanic_train_encoded.columns)
print("The number of missing values :\n", X_titanic_train_complete.isnull().sum())

The number of missing values :
 pclass    0
sex       0
age       0
sibsp     0
parch     0
fare      0
cabin     0
dtype: int64


In [ ]:
# Giá trị k mặc định cho bộ nhập KNN là 5, bạn có thể thay đổi như sau: 
imputer = KNNImputer(n_neighbors=2)
# etc etc...

## Iterative Imputer (`sklearn`)

Lưu ý đây là cách sklearn triển khai một phương thức ban đầu được gọi là "MICE" -- xem bài giảng 2 của tuần này để biết thêm chi tiết.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer()
X_titanic_train_complete = imputer.fit_transform(X_titanic_train_encoded)
# Đầu ra là 'numpy.ndarray' nên chúng tôi chuyển đổi nó thành khung dữ liệu để đảm bảo tính nhất quán
X_titanic_train_complete=pd.DataFrame(X_titanic_train_complete, columns=X_titanic_train_encoded.columns)
print("The number of missing values :\n", X_titanic_train_complete.isnull().sum())

The number of missing values :
 pclass    0
sex       0
age       0
sibsp     0
parch     0
fare      0
cabin     0
dtype: int64


Bạn có thể đặt lại các tham số mặc định của bộ nhập lặp. Ví dụ: bạn có thể đặt số lần lặp. Hơn nữa, bạn có thể chỉ định bộ ước lượng để ước lượng các giá trị còn thiếu.

In [ ]:
# Hãy sử dụng DT như một công cụ ước tính
from sklearn.tree import DecisionTreeRegressor
imputer = IterativeImputer(estimator=DecisionTreeRegressor())
X_titanic_train_complete = imputer.fit_transform(X_titanic_train_encoded)
# Đầu ra là 'numpy.ndarray' nên chúng tôi chuyển đổi nó thành khung dữ liệu để đảm bảo tính nhất quán
X_titanic_train_complete=pd.DataFrame(X_titanic_train_complete, columns=X_titanic_train_encoded.columns)
print("The number of missing values :\n", X_titanic_train_complete.isnull().sum())

The number of missing values :
 pclass    0
sex       0
age       0
sibsp     0
parch     0
fare      0
cabin     0
dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


## Áp dụng các mô hình đã học vào dữ liệu thử nghiệm chưa đầy đủ

Đầu tiên, áp dụng bộ mã hóa

In [ ]:
# Encoder_sex đã học phải được sử dụng để mã hóa dữ liệu thử nghiệm, LƯU Ý KHÔNG có sự phù hợp ở đây, chỉ cần chuyển đổi
X_titanic_test_encoded=X_titanic_test.copy()
X_titanic_test_encoded['sex'] = encoder_sex.transform(X_titanic_test_encoded['sex'].values.reshape(-1, 1))

#Bộ mã hóa đã học2 nên được sử dụng để mã hóa dữ liệu thử nghiệm, LƯU Ý KHÔNG có sự phù hợp ở đây, chỉ cần chuyển đổi
X_titanic_test_encoded['cabin'] = encoder_cabin.transform(X_titanic_test_encoded['cabin'].values.reshape(-1, 1).astype(str))
#Bây giờ, hãy lấy lại các giá trị nan bị thiếu trong dữ liệu được mã hóa
X_titanic_test_encoded['cabin'].replace(cabin_nan_code,np.nan,inplace=True)


Thứ hai, sử dụng bộ nhập đã học để ước tính các giá trị bị thiếu trong dữ liệu thử nghiệm

In [ ]:
print("The number of missing values in the test data before imputation :\n", X_titanic_test_encoded.isnull().sum())
X_titanic_test_complete = imputer.transform(X_titanic_test_encoded)
X_titanic_test_complete=pd.DataFrame(X_titanic_test_complete, columns=X_titanic_test_encoded.columns)
print("The number of missing values in the test data after imputation :\n", X_titanic_test_complete.isnull().sum())

The number of missing values in the test data before imputation :
 pclass      0
sex         0
age        76
sibsp       0
parch       0
fare        0
cabin     349
dtype: int64
The number of missing values in the test data after imputation :
 pclass    0
sex       0
age       0
sibsp     0
parch     0
fare      0
cabin     0
dtype: int64


Cuối cùng, chúng ta có thể thực hiện phân loại bằng cách sử dụng dữ liệu đầy đủ đã được suy ra.

In [ ]:
# Chúng ta sử dụng phép đo f vì các lớp không cân bằng
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state=0)
#classifier=SVC()
classifier.fit(X_titanic_train_complete, y_titanic_train)
print("F1 score after imputation = ", f1_score(classifier.predict(X_titanic_test_complete), y_titanic_test))

F1 score after imputation =  0.7289719626168224


----